# Comparing Models to Observations Using Datacubes

If necessary, install the DTCG API with:

```
pip install 'dtcg[oggm] @ git+https://github.com/DTC-Glaciers/dtcg'
```

In a cell below.

In [ ]:
import dtcg.integration.oggm_bindings as oggm_bindings
import dtcg.interface.plotting as dtcg_plotting
import holoviews as hv
import xarray as xr

## Downloading Data

Let's examine Bruarjökull. We can use DTCG to look up its RGI ID. It should be **RGI60-06.003377**.

In [ ]:
dtcg_oggm = oggm_bindings.BindingsCryotempo()
dtcg_oggm.init_oggm()

glaciers = dtcg_oggm.get_rgi_files_from_subregion(
    region_name="Iceland", subregion_name=""
)
rgi_id = dtcg_oggm.get_glacier_by_name(glaciers, "Bruarjökull").RGIId.values[0]
print(f"RGI ID: {rgi_id}")

We can download datacubes directly from a URL, and open them as data trees. This is equivalent to accessing a ``GeoZarrHandler``'s ``data_tree`` attribute.

In [ ]:
input_path = f"https://cluster.klima.uni-bremen.de/~dtcg/test_zarr/{rgi_id}.zarr/"
datacubes = xr.open_datatree(
    input_path, chunks={}, engine="zarr", consolidated=True, decode_cf=True
)

Let's take a look at 2015, [which we know from another notebook was an anomalous year for Iceland](https://notebooks.dtcglaciers.org//notebooks/04_calibrate_with_eolis_data.ipynb). We can compare the cumulative specific mass balance **observed by CryoSat-2** to the cumulative specific mass balance **modelled by OGGM**. To make things simpler, we also combine the data with some glacier attributes from OGGM. Again, we can do this using DTCG with just a few lines of code.

In [ ]:
gdir = dtcg_oggm.get_glacier_directories(rgi_ids=[rgi_id], from_prepro_level=4, prepro_border=80)[0]
runoff_data = dtcg_oggm.get_aggregate_runoff(gdir=gdir)

In [ ]:
graph_artist = dtcg_plotting.BokehGraph()
artist = dtcg_plotting.BokehCryotempo()
fig_smb_cumulative = artist.plot_eolis_smb(
    datacube=datacubes.L1, ref_year=2015, glacier_area=gdir.rgi_area_km2
).opts(title="a) Cumulative Specific Mass Balance (CryoSat)")
fig_runoff_cumulative = graph_artist.plot_runoff_timeseries(
    runoff=runoff_data["monthly_runoff"],
    ref_year=2015,
    cumulative=True,
).opts(title="b) Cumulative Monthly Runoff (OGGM)")
hv.Layout([fig_smb_cumulative, fig_runoff_cumulative]).opts(sizing_mode="stretch_width").opts(title="Bruarjökull, Iceland")

We can see from **Figure a)** that Bruarjökull experienced a significant delay to the start of the 2015 summer season, which led to the highest winter cumulative specific mass balance since at least 2011.
**Figure b)** shows a corresponding decrease in monthly runoff. Indeed, [Landsvirkjun considered the annual inflow into Vatnajökull to be "very dry"](https://gogn.lv.is/files/Arsskyrslur/LV-Annual_Report_2015.pdf). In the summer of 2015, Iceland experienced an exceptional hydrological anomaly that strained its hydropower system. Unusually cold weather and persistent cloud cover delayed and reduced glacier melt.
